In [35]:
import pandas as pd
import os
os.chdir("D:/functional-prediction/")

In [37]:
match_dict = {
    "haplotype_name": [],
    "variant": [],
    "sign": []
}

for fn in os.listdir("multi_var"):
    df_var = pd.read_csv(os.path.join("multi_var", fn), dtype=str).fillna("")
    
    for index, row in df_var.iterrows():
        variant = row["variant"]
        for col in df_var.columns:
            if col == "variant":
                continue
                
            haplotype = col
            sign = row[col]
            if "B1" in sign:
                match_dict["haplotype_name"].append(haplotype)
                match_dict["variant"].append(variant)
                match_dict["sign"].append(sign)

In [38]:
df_multi = pd.DataFrame(match_dict)

In [40]:
import sys
sys.path.append("D:\pgkb_graph")
os.chdir("D:\pgkb_graph")

from src.util.variant_mapping_util import variantMappingUtil

v = variantMappingUtil()

function_dict = {}
for gene, variant_dict in v.gene_functionality_dict.items():
    for key, value in variant_dict.items():
        if "*" in key:
            function_dict["{}{}".format(gene, key).strip()] = value["function_status"]
        else:
            function_dict["{} {}".format(gene, key).strip()] = value["function_status"]
            
os.chdir("D:/functional-prediction/")

In [41]:
df_d = pd.read_csv("output/dataset.csv", dtype=str).fillna("")
chr_dict = dict(zip(df_d["gene"].values, df_d["chr"].values))

In [42]:
import re
pos_regex = re.compile(r"^[\d]+")

info_dict = {
    "gene": [],
    "chr": [],
    "variant_start": [],
    "reference_allele": [],
    "variant_allele": [],
    "function": []
}

for index, row in df_multi.iterrows():
    haplotype = row["haplotype_name"]
    function = function_dict.get(haplotype, "")
    gene = haplotype.split("*")[0]
    chromo = chr_dict[gene]
    variant = row["variant"]
    if "ins" in variant or "del" in variant:
        variant_start = ""
        reference_allele = ""
        variant_allele = ""
    else:
        variant_start = re.findall(pos_regex, variant)[0]
        variant = variant.replace(variant_start, "").strip()
        reference_allele, variant_allele = variant.split(">")
        
    info_dict["gene"].append(gene)
    info_dict["chr"].append(chromo)
    info_dict["variant_start"].append(variant_start)
    info_dict["reference_allele"].append(reference_allele)
    info_dict["variant_allele"].append(variant_allele)
    info_dict["function"].append(function)

In [43]:
df_info = pd.DataFrame(info_dict)
df_multi = pd.concat([df_multi, df_info], axis=1)

In [44]:
df_multi

,haplotype_name,variant,sign,gene,chr,variant_start,reference_allele,variant_allele,function
0,CYP2B6*10,40991367A>T,B1,CYP2B6,19,40991367,A,T,Uncertain function
1,CYP2B6*17,40991381A>T,B1,CYP2B6,19,40991381,A,T,Normal function
2,CYP2B6*17,40991388A>G,B1,CYP2B6,19,40991388,A,G,Normal function
3,CYP2B6*17,40991390C>A,B1,CYP2B6,19,40991390,C,A,Normal function
4,CYP2B6*17,40991391G>C,B1,CYP2B6,19,40991391,G,C,Normal function
...,...,...,...,...,...,...,...,...,...
108,SLCO1B1*30,21178957A>G,B1,SLCO1B1,12,21178957,A,G,Uncertain function
109,SLCO1B1*39,21196975C>T,B1A,SLCO1B1,12,21196975,C,T,Uncertain function
110,SLCO1B1*47,21200544C>G,B1,SLCO1B1,12,21200544,C,G,No function
111,SLCO1B1*27,21202555C>G,B1,SLCO1B1,12,21202555,C,G,Uncertain function


In [45]:
df_multi["function"] = df_multi["function"].str.lower().str.strip()

df_multi = df_multi[(~df_multi["function"].str.contains("uncertain")) & (~df_multi["function"].str.contains("unknown"))]

In [50]:
os.chdir("D:/functional-prediction/")
df_multi[[
    "haplotype_name", "gene", "chr", "variant", 
    "variant_start", "reference_allele", 
    "variant_allele", "function"
]].to_csv("output/dataset_multi_var.csv", index=False)